# Pitch Type Classification
## Using Sklearn Random Forest
### Read Data

In [6]:
import json
import os

In [45]:
hits_folder_path = '/Users/tiffanynguyen/wsd-data/wsd-hits'
no_hits_folder_path = '/Users/tiffanynguyen/wsd-data/wsd-nohit'
pitch_type_data = []
pitch_path_data = []

for filename in os.listdir(hits_folder_path):
    if filename.endswith('.jsonl'):
        file_path = os.path.join(hits_folder_path, filename)
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            # only read data if it has both pitch type
            if(data['summary_acts']['pitch']['type']):
                pitch_type_data.append(data['summary_acts']['pitch']['type'])
                pitch_path_data.append(list((object['pos'] for object in data['samples_ball'])) )

#check that the length are the same (since they should be parallel lists)
print(len(pitch_type_data))
print(len(pitch_path_data))

42
42


### Train Model

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [101]:
#divide data
train_x, test_x, train_y, test_y = train_test_split(pitch_path_data, pitch_type_data, test_size=0.2, random_state=42)

In [102]:
#flatten
flattened_train_x = [sample.flatten() for sample in padded_train_x]
flattened_test_x = [sample.flatten() for sample in padded_test_x]

# pad
max_train_len = max(len(cur) for cur in flattened_train_x)
max_test_len = max(len(cur) for cur in flattened_test_x)
max_len = max(max_train_len, max_test_len)

flattened_train_x = [np.array(np.pad(sample, (0, max_len - len(sample)), mode='constant')) for sample in flattened_train_x]
flattened_test_x = [np.array(np.pad(sample, (0, max_len - len(sample)), mode='constant')) for sample in flattened_test_x]

print(len(flattened_train_x[0]))
print(len(flattened_train_x))
for i in range(len(flattened_train_x)):
    if len(flattened_train_x[i]) != 57600:
        print(i)

57600
33


In [103]:
# Feature scaling
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(flattened_train_x)
test_x_scaled = scaler.transform(flattened_test_x)

In [104]:
# Initialize and train the classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(flattened_train_x, train_y)

RandomForestClassifier(random_state=42)

In [105]:
pred_y = classifier.predict(flattened_test_x)

In [106]:
accuracy = accuracy_score(test_y, pred_y)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.22


### Plot Accuracy

In [107]:
import matplotlib.pyplot as plt